In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_path = "/content/drive/My Drive/plant-seedlings-classification/train"
test_path = "/content/drive/My Drive/plant-seedlings-classification/test"

In [ ]:
import os
import os,glob
import cv2
import numpy as np

In [ ]:
from sklearn import preprocessing as skLearnPreprocess
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
def getPath (path):
  return glob.glob(path)

paths = getPath(train_path +"/*")

##  Read Images

In [ ]:
images = []
classes=[]
missing=0
for class_folder_name in os.listdir(train_path):
    class_folder_path = os.path.join(train_path, class_folder_name)
    class_label = class_folder_name
    
    for image_path in glob.glob(os.path.join(class_folder_path, "*.png")):
        image_bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if image_bgr is None: # if the file contain any missing value ignore it
          missing += 1
          continue
        images.append(image_bgr)
        
        classes.append(class_label)

## Resize images

In [ ]:
# resize image
def resize_images(img):

  img = np.array(img).astype(np.uint8)
  #print(img.dtype)
  res = cv2.resize(img,(256,256), interpolation = cv2.INTER_CUBIC)
  return res

#save resized images into images.
images = [resize_images(img) for img in images]

In [ ]:
classes = np.array(classes).reshape(-1,1)

In [ ]:
images = np.array(images)

## Label Encoding

In [ ]:
def encodeLabels(y_train, y_valid):
  le = skLearnPreprocess.LabelEncoder()
  y_train = le.fit_transform(y_train)
  y_valid = le.fit_transform(y_valid)
  y_train = tf.keras.utils.to_categorical(y_train, num_classes=12)
  y_valid = tf.keras.utils.to_categorical(y_valid, num_classes=12)
  y_valid = np.array(y_valid)
  y_train = np.array(y_train)

  return y_train , y_valid

## Build Model

In [ ]:
def buildmodel():
    model = keras.Sequential([

    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomRotation(0.1),
    # First Convolutional Block
    layers.Conv2D(filters=32, kernel_size=3, activation="relu", padding='same',
                  # give the input dimensions in the first layer
                  # [height, width, color channels(RGB)]
                  input_shape=[256, 256, 3]),
    layers.MaxPool2D(),

    # Second Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),
    #layers.Dropout(0.2),

    # Third Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),
    #layers.Dropout(0.2),

    # Fourth Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),
    #layers.Dropout(0.2),

    # Fifth Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),
    #layers.Dropout(0.2),

    # Sixth Convolutional Block
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),
    #layers.Dropout(0.2),

    # Seventh Convolutional Block
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),
    #layers.Dropout(0.2),

    # Classifier Head
    layers.Flatten(),
    layers.Dense(units=128,kernel_regularizer=keras.regularizers.l2(0.01), activation="relu"),
    #layers.Dense(units=128,activation="relu"),
    layers.Dense(units=12, activation="softmax"),
   ])
    
    model.compile(
    optimizer= tf.keras.optimizers.Adam(epsilon=0.01),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
    )
    
    return(model)

## Early Stopping

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_categorical_accuracy', mode='max', min_delta=0.1,patience=10,
    restore_best_weights=True,
)

## Cross Validation

In [ ]:
kFold = StratifiedKFold(n_splits=5) 
accuracies = []
losses = []

In [ ]:
def train_evaluate(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 50, validation_data =(x_test,y_test), callbacks = [early_stopping])
    return model.evaluate(x_test, y_test)

In [ ]:
idx = 0
for train, test in kFold.split(images, classes):
  print("Starting Fold", idx)
  model = buildmodel()
  
  y_train, y_valid = encodeLabels(classes[train],classes[test])

  scores = train_evaluate(model, images[train], y_train, images[test],y_valid)
  accuracies.append(scores[1])
  losses.append (scores[0])
  print ("Validation accuracy: ", scores[1])
  print ("Validation loss", scores[0])

  idx += 1

Starting Fold 0


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/50


In [ ]:
print(accuracies)
print(np.mean(accuracies))
print(np.mean(losses))